In [2]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.97 🚀 Python-3.8.3 torch-1.13.1 CUDA:0 (NVIDIA A100 80GB PCIe, 81104MiB)
Setup complete ✅ (64 CPUs, 250.9 GB RAM, 12.0/50.0 GB disk)


In [2]:
from ultralytics import YOLO

# from roboflow import Roboflow
# rf = Roboflow(api_key="89hrk4aVWlA7jd5qmqTE")
# project = rf.workspace("").project("capseg")
# dataset = project.version(1).download("yolov5")


In [5]:
import time

In [59]:
t0 = time.time()
i=0
data_path = f'datasets_seg{i}' #the path you saved the augmented data for seg from roboflow
!mv $data_path datasets
!yolo task=detect mode=train model=yolov8x.pt data=./datasets/data.yaml epochs=100 imgsz=640 plots=True
!mv datasets $data_pat
!yolo task=segment mode=train model=yolov8x-seg.pt data=datasets/data.yaml epochs=100 imgsz=640
t1 = time.time()-t0

New https://pypi.org/project/ultralytics/8.0.101 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.97 🚀 Python-3.8.3 torch-1.13.1 CUDA:0 (NVIDIA A100 80GB PCIe, 81104MiB)
yolo/engine/trainer: task=segment, mode=train, model=yolov8x-seg.pt, data=datasets/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxe

In [77]:
# !pip install roboflow

# from roboflow import Roboflow

from roboflow import Roboflow
## deploy model 1

yourAPI=XXXX
Your_project_name = YYYY
rf = Roboflow(api_key=f"{yourAPI}")
project = rf.workspace().project(f"{Your_project_name}")





loading Roboflow workspace...
loading Roboflow project...


In [78]:
print(project)

{
  "name": "data_prep_seg1",
  "type": "instance-segmentation",
  "workspace": "capdetect"
}


In [79]:
model = project.version(1).model

In [80]:
# from roboflow import Roboflow

# rf = Roboflow(api_key="89hrk4aVWlA7jd5qmqTE")
# project = rf.workspace().project("data_prep2")
# model = project.version(1).model

In [81]:
import os

In [82]:
from statistics import mean
import torchvision
from tqdm import tqdm
from torch.nn.functional import threshold, normalize
from torchvision.ops import masks_to_boxes
from torchvision.utils import draw_bounding_boxes
from tqdm import tqdm
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [83]:
path = ''

In [84]:
red_path = path+'test/'
green_path = path+'Auto_green/'
label_path =  path+'test_label/'

In [85]:
red_files = os.listdir(red_path)
green_files = os.listdir(green_path)
if '.ipynb_checkpoints' in red_files:
    red_files.remove('.ipynb_checkpoints')

In [86]:
# red_files

In [87]:
save_path = f'output_new/end2end/{np.random.randint(10000)}/'

os.makedirs(save_path,exist_ok = True) 

In [88]:
# yolov8_seg/runs/segment/train/weights/best.pt

In [89]:
# !yolo task=segment mode=predict model=runs/segment/train/weights/best.pt conf=0.25 source=test/images save=true

In [90]:
# image_name = red_files[0].split('.')[0]
# image  = cv2.imread(red_path+image_name+'.jpg',-1)
# out = model.predict(red_path+image_name+'.jpg').json()

In [91]:
for i in range(len(red_files)):
    
    print(i)
    ### prompt-based
    image_name = red_files[i].split('.')[0] #test_name[i]
    image  = cv2.imread(red_path+image_name+'.jpg',-1)
    pred_label = {}
    pred_label['info'] = {}
    pred_label['info']['name'] = image_name+'.jpg'
    
    pred_label['info']['width'] = image.shape[0]
    pred_label['info']['height'] = image.shape[1]
    pred_label['info']['depth'] = image.shape[2]
    
    
    # image_name ='116_jpg.rf.36bd06d127f9d04216932cea04a15d48.jpg'
    
    out = model.predict(red_path+image_name+'.jpg').json()
    masks = []
    for predciton in out['predictions']:
        Object = {}
        x1,y1,x2,y2 = predciton['x']-predciton['width']/2,predciton['y']-predciton['height']/2,predciton['x']+predciton['width']/2,predciton['y']+predciton['height']/2
        input_box= np.array([x1,y1,x2,y2])
        Object['bbox'] =[x1,y1,x2,y2]
        cate = predciton['class']
        polygons_points = []
        
        for pair in predciton['points']:
            polygons_points.append([pair['x'],pair['y']])
            
        m = np.zeros(image.shape[:2])
        img_origin = m.copy()
        cv2.fillPoly(m,[np.array(polygons_points).astype(np.int32)], 1)
        m =  cv2.bitwise_xor(m, img_origin)    
        Object['area'] = np.sum(m) 
        Object['score'] =predciton['confidence']
        Object['segmentation'] = polygons_points
        Object['category'] =predciton['class']
        masks.append(Object)
        
    pred_label['objects'] = masks
    #save to file ...
   
    
    pred_label = json.dumps(pred_label)
    f = open(os.path.join(save_path+f'{image_name}.json'), 'w')
    f.write(pred_label)
    f.close()    
        
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
